<a href="https://colab.research.google.com/github/i-ganza007/Multimodal-Data-Preprocessing/blob/main/Product_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [13]:
customer_social_profiles = pd.read_excel('/content/Copy of customer_social_profiles.xlsx')
customer_transactions = pd.read_excel('/content/Copy of customer_transactions.xlsx')
customer_social_profiles.head()
customer_transactions.head()

,customer_id_legacy,transaction_id,purchase_amount,purchase_date,product_category,customer_rating
0,151,1001,408,2024-01-01,Sports,2.3
1,192,1002,332,2024-01-02,Electronics,4.2
2,114,1003,442,2024-01-03,Electronics,2.1
3,171,1004,256,2024-01-04,Clothing,2.8
4,160,1005,64,2024-01-05,Clothing,1.3


In [16]:
joined_set = customer_social_profiles.join(customer_transactions)
# joined_set.fillna(joined_set['customer_rating'].mean)
joined_set_filled = joined_set.fillna({'customer_rating': joined_set['customer_rating'].mean()})

In [26]:
# Columns like "customer_id_legacy", "transaction_id" aren't useful
final_set = joined_set_filled.drop(['customer_id_legacy', 'transaction_id','customer_id_new'], axis=1)
final_set.head()

,social_media_platform,engagement_score,purchase_interest_score,review_sentiment,purchase_amount,purchase_date,product_category,customer_rating
0,LinkedIn,74,4.9,Positive,408.0,2024-01-01,Sports,2.3
1,Twitter,82,4.8,Neutral,332.0,2024-01-02,Electronics,4.2
2,Facebook,96,1.6,Positive,442.0,2024-01-03,Electronics,2.1
3,Twitter,89,2.6,Positive,256.0,2024-01-04,Clothing,2.8
4,Twitter,92,2.3,Neutral,64.0,2024-01-05,Clothing,1.3


In [27]:
final_set["purchase_date"] = pd.to_datetime(final_set["purchase_date"])
final_set["purchase_month"] = final_set["purchase_date"].dt.month
final_set["purchase_day"] = final_set["purchase_date"].dt.day
final_set["purchase_weekday"] = final_set["purchase_date"].dt.weekday
final_set = final_set.drop(columns=["purchase_date"])
final_set.head()

,social_media_platform,engagement_score,purchase_interest_score,review_sentiment,purchase_amount,product_category,customer_rating,purchase_month,purchase_day,purchase_weekday
0,LinkedIn,74,4.9,Positive,408.0,Sports,2.3,1.0,1.0,0.0
1,Twitter,82,4.8,Neutral,332.0,Electronics,4.2,1.0,2.0,1.0
2,Facebook,96,1.6,Positive,442.0,Electronics,2.1,1.0,3.0,2.0
3,Twitter,89,2.6,Positive,256.0,Clothing,2.8,1.0,4.0,3.0
4,Twitter,92,2.3,Neutral,64.0,Clothing,1.3,1.0,5.0,4.0


In [28]:
le = LabelEncoder()
final_set["product_category"] = le.fit_transform(final_set["product_category"])
final_set['review_sentiment'] = le.fit_transform(final_set['review_sentiment'])
final_set['social_media_platform'] = le.fit_transform(final_set['social_media_platform'])
final_set.head()

,social_media_platform,engagement_score,purchase_interest_score,review_sentiment,purchase_amount,product_category,customer_rating,purchase_month,purchase_day,purchase_weekday
0,2,74,4.9,2,408.0,4,2.3,1.0,1.0,0.0
1,4,82,4.8,1,332.0,2,4.2,1.0,2.0,1.0
2,0,96,1.6,2,442.0,2,2.1,1.0,3.0,2.0
3,4,89,2.6,2,256.0,1,2.8,1.0,4.0,3.0
4,4,92,2.3,1,64.0,1,1.3,1.0,5.0,4.0


In [29]:
from sklearn.model_selection import train_test_split
X = final_set.drop("product_category", axis=1)
y = final_set["product_category"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
#Random Trees
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.22580645161290322


In [31]:
#XGBoost
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.1935483870967742


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [22:57:36] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [32]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(n_estimators=100, max_depth=4, learning_rate=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000128 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 179
[LightGBM] [Info] Number of data points in the train set: 124, number of used features: 9
[LightGBM] [Info] Start training from score -1.875843
[LightGBM] [Info] Start training from score -1.642228
[LightGBM] [Info] Start training from score -1.601406
[LightGBM] [Info] Start training from score -1.601406
[LightGBM] [Info] Start training from score -1.524445
[LightGBM] [Info] Start training from score -3.433987
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur